In [5]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras import layers, models
from tensorflow.keras.datasets import mnist
import matplotlib.pyplot as plt


In [6]:
# Load MNIST dataset
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

# Reshape images to (28, 28, 1) and resize to (32, 32, 3) for ResNet-50 input
train_images = tf.image.resize_with_pad(train_images[..., None], 32, 32)
test_images = tf.image.resize_with_pad(test_images[..., None], 32, 32)

# Convert grayscale (1-channel) to 3-channel to match ResNet-50 input format
train_images = tf.image.grayscale_to_rgb(train_images)
test_images = tf.image.grayscale_to_rgb(test_images)

# Normalize images to [0,1]
train_images, test_images = train_images / 255.0, test_images / 255.0

# Convert labels to categorical (one-hot encoding)
num_classes = 10
train_labels = tf.keras.utils.to_categorical(train_labels, num_classes)
test_labels = tf.keras.utils.to_categorical(test_labels, num_classes)

11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [3]:
def create_resnet50():
    # Load ResNet-50 model (pretrained on ImageNet, excluding top layers)
    base_model = ResNet50(weights=None, include_top=False, input_shape=(32, 32, 3))

    # Freeze the base model (optional)
    # base_model.trainable = False

    # Add custom layers for MNIST classification
    model = models.Sequential([
        base_model,
        layers.GlobalAveragePooling2D(),
        layers.Dense(128, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(num_classes, activation='softmax')  # Output layer (10 classes)
    ])

    # Compile the model
    model.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

    return model

In [ ]:
# Create ResNet-50 model
resnet_model = create_resnet50()

# Train Model
history = resnet_model.fit(train_images, train_labels, epochs=10, batch_size=64, validation_split=0.2)


Epoch 1/10
  3/750 ━━━━━━━━━━━━━━━━━━━━ 50:17 4s/step - accuracy: 0.1319 - loss: 4.4680

In [ ]:
# Evaluate the model
test_loss, test_acc = resnet_model.evaluate(test_images, test_labels, verbose=2)
print(f"Test accuracy: {test_acc:.4f}")


In [ ]:
# Plot training history
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Test Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.title('Accuracy Over Epochs')

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Test Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.title('Loss Over Epochs')

plt.show()